<a href="https://colab.research.google.com/github/nghiemkhoa1235-boop/mafbj/blob/main/ProjectAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications.efficientnet import preprocess_input
import numpy as np
import cv2
from google.colab import files
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')

# ===============================
# 🚀 LOAD MODEL AND CLASSES
# ===============================

print("🚀 LOADING BEST MODEL FOR PREDICTION")

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)
data_root = '/content/drive/MyDrive/Project_AI_Ki_3/AI/do_an'
backup_dir = os.path.join(data_root, 'training_backups')

# Load the final best model
model_path = os.path.join(backup_dir, 'FINAL_BEST_FOOD_MODEL.keras')
if not os.path.exists(model_path):
    # Fallback to previous best if not found
    model_path = os.path.join(backup_dir, 'ULTIMATE_FOOD_RECOGNITION_MODEL.keras')
    if not os.path.exists(model_path):
        print("❌ Model not found! Check Drive path.")
        exit()

model = tf.keras.models.load_model(model_path)
print(f"✅ Model loaded: {model_path}")

# Classes from training (fixed from your dataset)
classes = ['ca_hu_kho', 'canh_chua_co_ca', 'canh_chua_khong_ca', 'canh_rau', 'com',
           'dau_hu_sot_ca', 'khay', 'rau_xao', 'suon_nuong', 'thit_kho',
           'thit_kho_trung', 'trung_chien']
num_classes = len(classes)
print(f"📋 Classes: {classes}")

# ===============================
# 🖼️ UPLOAD AND PREDICT IMAGE
# ===============================

print("\n📁 Upload a new food image (any JPG/PNG, not from training data):")
uploaded = files.upload()

for filename in uploaded.keys():
    # Load and preprocess image
    img = cv2.imread(filename)
    if img is None:
        print(f"❌ Cannot load image: {filename}")
        continue

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, (300, 300))  # Target size from training
    img_array = preprocess_input(img_resized.astype(np.float32)).reshape(1, 300, 300, 3)

    # Predict
    predictions = model.predict(img_array, verbose=0)
    predicted_class_idx = np.argmax(predictions[0])
    confidence = predictions[0][predicted_class_idx]
    predicted_class = classes[predicted_class_idx]

    print(f"\n🎯 PREDICTION FOR '{filename}':")
    print(f"   Món ăn: {predicted_class}")
    print(f"   Độ tin cậy: {confidence:.4f} ({confidence*100:.2f}%)")


print("\n✅ PREDICTION COMPLETE! Use any new food photo for testing.")



🚀 LOADING BEST MODEL FOR PREDICTION
Mounted at /content/drive
✅ Model loaded: /content/drive/MyDrive/Project_AI_Ki_3/AI/do_an/training_backups/ULTIMATE_FOOD_RECOGNITION_MODEL.keras
📋 Classes: ['ca_hu_kho', 'canh_chua_co_ca', 'canh_chua_khong_ca', 'canh_rau', 'com', 'dau_hu_sot_ca', 'khay', 'rau_xao', 'suon_nuong', 'thit_kho', 'thit_kho_trung', 'trung_chien']

📁 Upload a new food image (any JPG/PNG, not from training data):


Saving 129.jpg to 129.jpg

🎯 PREDICTION FOR '129.jpg':
   Món ăn: rau_xao
   Độ tin cậy: 0.9681 (96.81%)

✅ PREDICTION COMPLETE! Use any new food photo for testing.


In [ ]:
from google.colab import drive
from IPython.display import clear_output, HTML, display
import warnings
warnings.filterwarnings('ignore')

import os
import threading
import time
from tensorflow.keras.models import load_model
import cv2
import numpy as np
from fastapi import FastAPI, UploadFile, File
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
import uvicorn
import base64
import tensorflow as tf
from tensorflow.keras.applications.efficientnet import preprocess_input
import random
import socket

# Install pyngrok
!pip install pyngrok -q

from pyngrok import ngrok

# ============= BƯỚC 1: MOUNT DRIVE VÀ LOAD MODEL =============
def mount_drive_with_retry(max_attempts=3, delay=5):
    """Thử mount Google Drive với cơ chế retry."""
    for attempt in range(max_attempts):
        try:
            if not os.path.exists('/content/drive/MyDrive'):
                drive.mount('/content/drive', force_remount=True)
                clear_output(wait=True)
                print("Google Drive mounted successfully.")
                return True
            else:
                print("Google Drive already mounted.")
                return True
        except Exception as e:
            print(f"⚠️ Mount attempt {attempt + 1}/{max_attempts} failed: {e}")
            if attempt < max_attempts - 1:
                time.sleep(delay)
    print("❌ Failed to mount Google Drive after retries.")
    return False

# Thử mount Drive với retry
if not mount_drive_with_retry():
    print("⚠️ Cannot proceed without Google Drive. Please check your connection and try again.")
    model = None
else:
    data_root = '/content/drive/MyDrive/Project_AI_Ki_3/AI/do_an'
    backup_dir = os.path.join(data_root, 'training_backups')

    # Kiểm tra sự tồn tại của file mô hình
    model_path = os.path.join(backup_dir, 'FINAL_BEST_FOOD_MODEL.keras')
    fallback_model_path = os.path.join(backup_dir, 'ULTIMATE_FOOD_RECOGNITION_MODEL.keras')

    if os.path.exists(model_path):
        try:
            model = load_model(model_path)
            print(f"Model loaded successfully from {model_path}")
        except Exception as e:
            print(f"⚠️ Error loading model from {model_path}: {e}")
            model = None
    elif os.path.exists(fallback_model_path):
        try:
            model = load_model(fallback_model_path)
            print(f"Fallback model loaded successfully from {fallback_model_path}")
        except Exception as e:
            print(f"⚠️ Error loading fallback model from {fallback_model_path}: {e}")
            model = None
    else:
        print("⚠️ No model files found at specified paths. Using random prediction mode.")
        model = None

classes = ['ca_hu_kho', 'canh_chua_co_ca', 'canh_chua_khong_ca', 'canh_rau', 'com',
           'dau_hu_sot_ca', 'khay', 'rau_xao', 'suon_nuong', 'thit_kho',
           'thịt kho 1 trứng', 'thịt kho 2 trứng', 'trung_chien']

class_to_food = {
    'com': {'name': 'Cơm Trắng', 'price': 10000},
    'dau_hu_sot_ca': {'name': 'Đậu Hũ Sốt Cà', 'price': 25000},
    'ca_hu_kho': {'name': 'Cá Hũ Kho', 'price': 30000},
    'thit_kho': {'name': 'Thịt Kho', 'price': 30000},
    'thịt kho 1 trứng': {'name': 'Thịt Kho 1 Trứng', 'price': 36000},
    'thịt kho 2 trứng': {'name': 'Thịt Kho 2 Trứng', 'price': 42000},
    'canh_chua_co_ca': {'name': 'Canh Chua Có Cà', 'price': 25000},
    'canh_chua_khong_ca': {'name': 'Canh Chua Không Cà', 'price': 10000},
    'suon_nuong': {'name': 'Sườn Nướng', 'price': 30000},
    'canh_rau': {'name': 'Canh Rau', 'price': 7000},
    'rau_xao': {'name': 'Rau Xào', 'price': 10000},
    'trung_chien': {'name': 'Trứng Chiên', 'price': 25000},
    'khay': {'name': 'Không xác định', 'price': 0}  # Xử lý khay rỗng
}

regions = {
    "Ô 1 (trên trái)": (50, 50, 400, 290),
    "Ô 2 (trên giữa)": (450, 50, 800, 290),
    "Ô 3 (trên phải)": (850, 50, 1200, 290),
    "Ô 4 (dưới trái)": (100, 350, 550, 650),
    "Ô 5 (dưới phải)": (650, 350, 1150, 650),
}

def predict_food(crop_img):
    if model is None:
        random_class = random.choice(list(class_to_food.keys()))
        food_info = class_to_food[random_class]
        confidence = random.uniform(0.85, 0.99)
        return food_info['name'], confidence, food_info['price']

    try:
        # Sửa lỗi: Kiểm tra ảnh đầu vào
        if crop_img is None or crop_img.size == 0:
            return "Không xác định", 0.0, 0

        img_rgb = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
        img_resized = cv2.resize(img_rgb, (300, 300))

        # Sửa lỗi: Đảm bảo kiểu dữ liệu phù hợp
        if img_resized.dtype != np.float32:
            img_resized = img_resized.astype(np.float32)

        img_array = preprocess_input(img_resized).reshape(1, 300, 300, 3)

        predictions = model.predict(img_array, verbose=0)
        predicted_class_idx = np.argmax(predictions[0])
        confidence = float(predictions[0][predicted_class_idx])
        predicted_class = classes[predicted_class_idx]

        food_info = class_to_food.get(predicted_class, {'name': 'Không xác định', 'price': 0})
        return food_info['name'], confidence, food_info['price']
    except Exception as e:
        print(f"⚠️ Lỗi predict: {e}")
        return "Không xác định", 0.0, 0

def crop_to_base64(crop_img):
    try:
        # Sửa lỗi: Kiểm tra ảnh trước khi encode
        if crop_img is None or crop_img.size == 0:
            # Trả về ảnh trắng nếu crop rỗng
            blank_image = np.ones((100, 100, 3), dtype=np.uint8) * 255
            _, buffer = cv2.imencode('.jpg', blank_image)
        else:
            _, buffer = cv2.imencode('.jpg', crop_img)

        return base64.b64encode(buffer).decode('utf-8')
    except Exception as e:
        print(f"⚠️ Lỗi base64: {e}")
        # Trả về ảnh trắng dự phòng
        blank_image = np.ones((100, 100, 3), dtype=np.uint8) * 255
        _, buffer = cv2.imencode('.jpg', blank_image)
        return base64.b64encode(buffer).decode('utf-8')

def is_port_in_use(port):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(('localhost', port)) == 0

PORT = 8001
if is_port_in_use(PORT):
    PORT = 8002

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/detect_tray")
async def detect_tray(file: UploadFile = File(...)):
    try:
        contents = await file.read()

        # Sửa lỗi: Kiểm tra file có dữ liệu không
        if not contents:
            return JSONResponse(content={"error": "File trống"}, status_code=400)

        nparr = np.frombuffer(contents, np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        if img is None:
            return JSONResponse(content={"error": "Không đọc được ảnh"}, status_code=400)

        fixed_w, fixed_h = 1280, 720
        img = cv2.resize(img, (fixed_w, fixed_h))

        results = []
        total_price = 0

        for idx, (name, (x1, y1, x2, y2)) in enumerate(regions.items(), 1):
            try:
                # Sửa lỗi: Đảm bảo coordinates hợp lệ
                x1, y1, x2, y2 = max(0, x1), max(0, y1), min(fixed_w, x2), min(fixed_h, y2)

                if x2 <= x1 or y2 <= y1:
                    continue

                crop = img[y1:y2, x1:x2]

                # Sửa lỗi: Kiểm tra crop có dữ liệu không
                if crop is None or crop.size == 0:
                    food_name, confidence, price = "Không xác định", 0.0, 0
                else:
                    food_name, confidence, price = predict_food(crop)

                total_price += price
                crop_base64 = crop_to_base64(crop)

                results.append({
                    'id': idx,
                    'region': name,
                    'food_name': food_name,
                    'confidence': round(confidence * 100, 1),
                    'price': price,
                    'image': f"data:image/jpeg;base64,{crop_base64}"
                })

            except Exception as e:
                print(f"⚠️ Lỗi region {name}: {e}")
                continue

        return {"success": True, "results": results, "total_price": total_price}

    except Exception as e:
        return JSONResponse(content={"error": str(e)}, status_code=500)

@app.post("/detect_single")
async def detect_single(file: UploadFile = File(...)):
    try:
        contents = await file.read()

        # Sửa lỗi: Kiểm tra file có dữ liệu không
        if not contents:
            return JSONResponse(content={"error": "File trống"}, status_code=400)

        nparr = np.frombuffer(contents, np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        if img is None:
            return JSONResponse(content={"error": "Không đọc được ảnh"}, status_code=400)

        food_name, confidence, price = predict_food(img)
        crop_base64 = crop_to_base64(img)

        result = {
            'id': 1,
            'region': 'Món đơn',
            'food_name': food_name,
            'confidence': round(confidence * 100, 1),
            'price': price,
            'image': f"data:image/jpeg;base64,{crop_base64}"
        }

        return {"success": True, "results": [result], "total_price": price}

    except Exception as e:
        return JSONResponse(content={"error": str(e)}, status_code=500)

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=PORT, log_level="error")

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()
time.sleep(3)

# Set up ngrok - Sửa lỗi authentication
try:
    ngrok.set_auth_token("YOUR_NGROK_AUTHTOKEN_HERE")
except Exception as e:
    print(f"⚠️ Lỗi ngrok auth: {e}")

# Create ngrok tunnel
try:
    tunnel = ngrok.connect(PORT, "http")
    public_url = tunnel.public_url
    print(f"Public URL: {public_url}")
except Exception as e:
    print(f"⚠️ Lỗi kết nối ngrok: {e}")
    public_url = f"http://localhost:{PORT}"

clear_output(wait=True)

# ============= BƯỚC 3: GIAO DIỆN HTML =============
html_code = f"""
<!DOCTYPE html>
<html lang="vi">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>App Đặt Đồ Ăn - FoodApp (Genshin Style)</title>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700&display=swap" rel="stylesheet">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css">
    <style>
        * {{
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }}
        body {{
            font-family: 'Inter', sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 50%, #f093fb 100%);
            color: #ffffff;
            line-height: 1.5;
            display: flex;
            flex-direction: column;
            height: 100vh;
            overflow: hidden;
            position: relative;
        }}
        body::before {{
            content: '';
            position: absolute;
            top: 0;
            left: 0;
            right: 0;
            bottom: 0;
            background: url('data:image/svg+xml,<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 100 100"><defs><radialGradient id="a" cx="50%" cy="50%" r="50%"><stop offset="0%" stop-color="%23ffffff" stop-opacity="0.1"/><stop offset="100%" stop-color="%23ffffff" stop-opacity="0"/></radialGradient></defs><circle cx="20" cy="20" r="2" fill="url(%23a)"/><circle cx="80" cy="80" r="1.5" fill="url(%23a)"/><circle cx="50" cy="10" r="1" fill="url(%23a)"/></svg>') repeat;
            pointer-events: none;
            z-index: 0;
        }}
        .header {{
            background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);
            color: white;
            text-align: center;
            padding: 20px 0;
            box-shadow: 0 8px 32px rgba(79, 172, 254, 0.3);
            position: relative;
            z-index: 10;
            backdrop-filter: blur(10px);
        }}
        .logo {{
            font-size: 32px;
            font-weight: 700;
            display: flex;
            align-items: center;
            justify-content: center;
            gap: 15px;
            text-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }}
        .logo i {{ font-size: 40px; animation: glow 2s ease-in-out infinite alternate; }}
        @keyframes glow {{
            from {{ text-shadow: 0 0 5px #00f2fe; }}
            to {{ text-shadow: 0 0 20px #00f2fe, 0 0 30px #4facfe; }}
        }}

        .main-content {{
            flex: 1;
            overflow-y: auto;
            padding: 20px;
            margin-bottom: 80px;
            position: relative;
            z-index: 1;
        }}

        .tabs {{
            position: fixed;
            bottom: 0;
            left: 0;
            right: 0;
            background: rgba(79, 172, 254, 0.9);
            backdrop-filter: blur(20px);
            display: flex;
            justify-content: space-around;
            padding: 10px 0;
            box-shadow: 0 -8px 32px rgba(79, 172, 254, 0.3);
            z-index: 100;
        }}
        .tab-btn {{
            display: flex;
            flex-direction: column;
            align-items: center;
            background: none;
            border: none;
            font-size: 11px;
            font-weight: 500;
            color: rgba(255,255,255,0.8);
            cursor: pointer;
            transition: all 0.3s ease;
            padding: 8px 12px;
            flex: 1;
            border-radius: 20px 20px 0 0;
        }}
        .tab-btn i {{ font-size: 24px; margin-bottom: 3px; }}
        .tab-btn.active {{
            color: #ffffff;
            background: rgba(255,255,255,0.2);
            transform: translateY(-5px);
        }}
        .tab-btn:hover {{ color: #ffffff; background: rgba(255,255,255,0.1); }}

        .tab-content {{
            display: none;
            animation: fadeInMagic 0.6s ease-out;
            min-height: 100%;
        }}
        .tab-content.active {{ display: block; }}
        @keyframes fadeInMagic {{
            from {{ opacity: 0; transform: scale(0.95) translateY(20px); filter: blur(5px); }}
            to {{ opacity: 1; transform: scale(1) translateY(0); filter: blur(0); }}
        }}

        /* Home Tab - Hero Banner + Floating cards */
        .hero-banner {{
            background: linear-gradient(135deg, rgba(118, 75, 162, 0.8) 0%, rgba(240, 147, 251, 0.8) 100%);
            backdrop-filter: blur(10px);
            border-radius: 25px;
            padding: 40px 30px;
            text-align: center;
            margin-bottom: 30px;
            box-shadow: 0 15px 50px rgba(118, 75, 162, 0.3);
            border: 1px solid rgba(255,255,255,0.2);
            position: relative;
            overflow: hidden;
        }}
        .hero-banner::before {{
            content: '';
            position: absolute;
            top: -50%;
            left: -50%;
            width: 200%;
            height: 200%;
            background: radialGradient(circle, rgba(0, 242, 254, 0.1) 0%, transparent 70%);
            animation: rotate 20s linear infinite;
        }}
        @keyframes rotate {{
            0% {{ transform: rotate(0deg); }}
            100% {{ transform: rotate(360deg); }}
        }}
        .hero-title {{
            font-size: 36px;
            font-weight: 700;
            margin-bottom: 10px;
            text-shadow: 0 2px 4px rgba(0,0,0,0.3);
            animation: pulse 2s ease-in-out infinite alternate;
        }}
        @keyframes pulse {{
            from {{ transform: scale(1); }}
            to {{ transform: scale(1.05); }}
        }}
        .hero-subtitle {{
            font-size: 18px;
            margin-bottom: 25px;
            opacity: 0.9;
        }}
        .hero-cta {{
            background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);
            color: white;
            padding: 15px 40px;
            border-radius: 30px;
            font-size: 16px;
            font-weight: 600;
            cursor: pointer;
            border: none;
            transition: all 0.3s;
            box-shadow: 0 6px 20px rgba(79, 172, 254, 0.4);
            text-decoration: none;
            display: inline-block;
        }}
        .hero-cta:hover {{
            transform: translateY(-3px);
            box-shadow: 0 12px 30px rgba(79, 172, 254, 0.5);
        }}
        .home-grid {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(150px, 1fr));
            gap: 20px;
            margin-bottom: 30px;
        }}
        .home-card {{
            background: rgba(255,255,255,0.1);
            backdrop-filter: blur(10px);
            border-radius: 20px;
            padding: 25px 15px;
            text-align: center;
            box-shadow: 0 8px 32px rgba(0,0,0,0.1);
            cursor: pointer;
            transition: all 0.4s ease;
            border: 1px solid rgba(255,255,255,0.2);
            position: relative;
            overflow: hidden;
            animation: fadeIn 0.5s ease-in;
        }}
        @keyframes fadeIn {{
            from {{ opacity: 0; }}
            to {{ opacity: 1; }}
        }}
        .home-card::before {{
            content: '';
            position: absolute;
            top: 0;
            left: -100%;
            width: 100%;
            height: 100%;
            background: linear-gradient(90deg, transparent, rgba(255,255,255,0.2), transparent);
            transition: left 0.5s;
        }}
        .home-card:hover::before {{ left: 100%; }}
        .home-card:hover {{
            transform: translateY(-10px) rotateX(5deg);
            box-shadow: 0 20px 40px rgba(79, 172, 254, 0.3);
            background: rgba(255,255,255,0.2);
        }}
        .home-card img {{ width: 60px; height: 60px; border-radius: 15px; margin-bottom: 15px; box-shadow: 0 4px 15px rgba(0,0,0,0.2); }}
        .home-card h3 {{ font-size: 16px; margin-bottom: 8px; color: #ffffff; font-weight: 600; text-shadow: 0 1px 2px rgba(0,0,0,0.5); }}
        .home-card p {{ font-size: 12px; color: rgba(255,255,255,0.9); }}

        /* Nhận diện Tab - Mystical upload area */
        .recog-section {{
            display: flex;
            flex-direction: column;
            gap: 25px;
            padding: 0;
        }}
        .upload-area {{
            background: rgba(255,255,255,0.1);
            backdrop-filter: blur(10px);
            border: 2px dashed rgba(255,255,255,0.5);
            border-radius: 25px;
            padding: 40px;
            text-align: center;
            cursor: pointer;
            transition: all 0.4s ease;
            box-shadow: 0 8px 32px rgba(0,0,0,0.1);
            border: 1px solid rgba(255,255,255,0.2);
        }}
        .upload-area:hover {{
            background: rgba(255,255,255,0.2);
            border-color: #00f2fe;
            transform: scale(1.02);
            box-shadow: 0 15px 40px rgba(0, 242, 254, 0.3);
        }}
        .upload-area input {{ display: none; }}
        .upload-btn {{
            background: linear-gradient(135deg, #4facfe, #00f2fe);
            color: white;
            padding: 15px 25px;
            border-radius: 30px;
            cursor: pointer;
            font-size: 14px;
            border: none;
            margin: 8px;
            font-weight: 600;
            transition: all 0.3s;
            box-shadow: 0 4px 15px rgba(79, 172, 254, 0.3);
        }}
        .upload-btn:hover {{ transform: translateY(-2px); box-shadow: 0 8px 25px rgba(79, 172, 254, 0.4); }}
        .camera-btn {{
            background: linear-gradient(135deg, #764ba2, #f093fb);
            color: white;
            padding: 15px 25px;
            border-radius: 30px;
            cursor: pointer;
            margin-top: 15px;
            font-size: 14px;
            border: none;
            font-weight: 600;
            box-shadow: 0 4px 15px rgba(118, 75, 162, 0.3);
        }}
        .camera-preview {{
            display: none;
            text-align: center;
            background: rgba(255,255,255,0.1);
            backdrop-filter: blur(10px);
            border-radius: 25px;
            overflow: hidden;
            box-shadow: 0 8px 32px rgba(0,0,0,0.1);
        }}
        .camera-preview.active {{ display: block; }}
        #cameraFeed {{
            width: 100%;
            max-height: 350px;
            border-radius: 0;
        }}
        .loading {{
            display: none;
            text-align: center;
            padding: 50px 30px;
            background: rgba(255,255,255,0.1);
            backdrop-filter: blur(10px);
            border-radius: 25px;
            box-shadow: 0 8px 32px rgba(0,0,0,0.1);
        }}
        .loading.active {{ display: block; }}
        .spinner {{
            border: 4px solid rgba(255,255,255,0.2);
            border-top: 4px solid #00f2fe;
            border-radius: 50%;
            width: 60px;
            height: 60px;
            animation: spin 1s linear infinite;
            margin: 0 auto 20px;
        }}
        @keyframes spin {{
            0% {{ transform: rotate(0deg); }}
            100% {{ transform: rotate(360deg); }}
        }}

        .results-grid {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(140px, 1fr));
            gap: 20px;
            margin-top: 25px;
            max-height: 300px;
            overflow-y: auto; /* Thêm thanh cuộn cho kết quả nhận diện */
        }}
        .result-item {{
            background: rgba(255,255,255,0.1);
            backdrop-filter: blur(10px);
            border-radius: 20px;
            padding: 20px;
            text-align: center;
            box-shadow: 0 8px 32px rgba(0,0,0,0.1);
            transition: all 0.4s ease;
            border: 1px solid rgba(255,255,255,0.2);
        }}
        .result-item:hover {{
            transform: translateY(-8px) rotate(1deg);
            box-shadow: 0 20px 40px rgba(0, 242, 254, 0.3);
            background: rgba(255,255,255,0.2);
        }}
        .result-item img {{ width: 100%; height: 90px; object-fit: cover; border-radius: 15px; margin-bottom: 15px; }}
        .result-label {{ font-weight: 600; margin-bottom: 8px; color: #ffffff; font-size: 13px; text-shadow: 0 1px 2px rgba(0,0,0,0.5); }}
        .result-conf {{ font-size: 11px; color: #f093fb; margin-bottom: 8px; }}
        .result-price {{ font-weight: 700; color: #00f2fe; font-size: 15px; text-shadow: 0 1px 2px rgba(0,0,0,0.5); }}

        .summary-box {{
            background: linear-gradient(135deg, #764ba2 0%, #f093fb 100%);
            color: white;
            padding: 25px;
            border-radius: 25px;
            margin: 25px 0;
            text-align: center;
            box-shadow: 0 12px 40px rgba(118, 75, 162, 0.3);
            backdrop-filter: blur(10px);
        }}
        .summary-total {{
            font-size: 28px;
            font-weight: 700;
            margin: 12px 0;
            text-shadow: 0 2px 4px rgba(0,0,0,0.3);
        }}
        .checkout-btn {{
            width: 100%;
            padding: 18px;
            background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);
            color: white;
            border: none;
            border-radius: 30px;
            font-size: 18px;
            font-weight: 600;
            cursor: pointer;
            margin-top: 20px;
            transition: all 0.3s;
            box-shadow: 0 6px 20px rgba(79, 172, 254, 0.4);
        }}
        .checkout-btn:hover {{ transform: translateY(-3px); box-shadow: 0 12px 30px rgba(79, 172, 254, 0.5); }}

        .qr-section {{
            display: none;
            text-align: center;
            padding: 30px;
            background: rgba(255,255,255,0.1);
            backdrop-filter: blur(10px);
            border-radius: 25px;
            margin-top: 25px;
            box-shadow: 0 8px 32px rgba(0,0,0,0.1);
        }}
        .qr-section.active {{ display: block; }}
        .qr-section img {{ width: 200px; height: 200px; border-radius: 20px; margin-bottom: 20px; box-shadow: 0 8px 25px rgba(0,0,0,0.2); }}

        .guide-section {{
            background: rgba(255,255,255,0.1);
            backdrop-filter: blur(10px);
            border-radius: 25px;
            padding: 30px;
            text-align: center;
            margin-top: 20px;
            box-shadow: 0 8px 32px rgba(0,0,0,0.1);
            border: 1px solid rgba(255,255,255,0.2);
        }}
        .guide-section h3 {{
            font-size: 20px;
            margin-bottom: 15px;
            color: #00f2fe;
            text-shadow: 0 1px 2px rgba(0,0,0,0.5);
        }}
        .guide-section p {{
            font-size: 14px;
            margin-bottom: 20px;
            color: rgba(255,255,255,0.9);
        }}
        .guide-section img {{
            width: 100%;
            max-width: 300px;
            border-radius: 15px;
            box-shadow: 0 4px 15px rgba(0,0,0,0.2);
        }}

        /* Menu Tab - Enhanced with larger cards and hover expansion */
        .menu-search {{
            width: 100%;
            padding: 18px;
            border-radius: 30px;
            border: 1px solid rgba(255,255,255,0.3);
            margin-bottom: 25px;
            background: rgba(255,255,255,0.1);
            backdrop-filter: blur(10px);
            color: #ffffff;
            font-size: 15px;
            box-shadow: 0 4px 15px rgba(0,0,0,0.1);
        }}
        .menu-container {{
            display: flex;
            height: calc(100% - 50px);  /* Adjust for search bar */
        }}
        .menu-left {{
            flex: 2;
            overflow-y: auto;
            padding-right: 20px;
            display: flex;
            flex-direction: column;
            gap: 20px;
            max-height: calc(100vh - 250px); /* Giữ kích thước cố định và thêm thanh cuộn để hiển thị hết các món ăn */
        }}
        .menu-card {{
            background: rgba(255,255,255,0.1);
            backdrop-filter: blur(10px);
            border-radius: 25px;
            overflow: hidden;
            box-shadow: 0 8px 32px rgba(0,0,0,0.1);
            cursor: pointer;
            transition: all 0.4s ease;
            border: 1px solid rgba(255,255,255,0.2);
            display: flex;
            align-items: center;
            padding: 15px;
            min-height: 120px; /* Giữ kích thước cố định cho mỗi card */
        }}
        .menu-card:hover {{
            transform: translateY(-5px);
            box-shadow: 0 15px 40px rgba(240, 147, 251, 0.3);
            background: rgba(255,255,255,0.2);
        }}
        .menu-card img {{
            width: 100px;
            height: 100px;
            object-fit: cover;
            border-radius: 15px;
            margin-right: 15px;
        }}
        .menu-card-info {{
            flex: 1;
        }}
        .menu-card h3 {{ font-size: 16px; margin-bottom: 5px; color: #ffffff; font-weight: 600; }}
        .menu-card .price {{
            color: #00f2fe;
            font-weight: 700;
            font-size: 14px;
        }}
        .detail-panel {{
            flex: 1;
            background: rgba(255,255,255,0.15);
            backdrop-filter: blur(10px);
            border-radius: 25px;
            padding: 20px;
            margin-left: 20px;
            overflow-y: auto;
            border: 1px solid rgba(255,255,255,0.2);
            box-shadow: 0 8px 32px rgba(0,0,0,0.1);
            display: flex;
            flex-direction: column;
        }}
        .detail-panel img {{
            width: 100%;
            height: 200px;
            object-fit: cover;
            border-radius: 15px;
            margin-bottom: 20px;
        }}
        .detail-table {{
            width: 100%;
            border-collapse: collapse;
            margin-bottom: 20px;
        }}
        .detail-table th, .detail-table td {{
            padding: 10px;
            border: 1px solid rgba(255,255,255,0.3);
            text-align: left;
        }}
        .detail-table th {{
            background: rgba(255,255,255,0.1);
            font-weight: 600;
        }}
        .detail-story {{
            font-size: 14px;
            line-height: 1.6;
        }}
        .detail-add-btn {{
            background: linear-gradient(135deg, #4facfe, #00f2fe);
            color: white;
            border: none;
            padding: 12px 24px;
            border-radius: 25px;
            font-size: 14px;
            cursor: pointer;
            margin-top: 20px;
            transition: all 0.3s;
            box-shadow: 0 4px 15px rgba(79, 172, 254, 0.3);
            align-self: center;
        }}
        .detail-add-btn:hover {{
            transform: translateY(-2px);
            box-shadow: 0 8px 25px rgba(79, 172, 254, 0.4);
        }}
        .cart-list, .history-list {{ list-style: none; padding: 0; overflow-y: auto; max-height: 400px; /* Thêm thanh cuộn cho giỏ hàng và lịch sử */ }}
        .cart-item, .history-item {{
            background: rgba(255,255,255,0.1);
            backdrop-filter: blur(10px);
            border-radius: 20px;
            padding: 20px;
            margin-bottom: 20px;
            box-shadow: 0 8px 32px rgba(0,0,0,0.1);
            display: flex;
            align-items: center;
            border: 1px solid rgba(255,255,255,0.2);
        }}
        .cart-item img {{ width: 60px; height: 60px; border-radius: 15px; margin-right: 20px; box-shadow: 0 4px 15px rgba(0,0,0,0.2); }}
        .cart-item-info {{ flex: 1; font-size: 14px; }}
        .cart-item .price {{ color: #00f2fe; font-weight: 600; font-size: 15px; }}
        .qty-controls {{
            display: flex;
            align-items: center;
            gap: 10px;
            margin-top: 10px;
        }}
        .qty-btn {{
            background: linear-gradient(135deg, #f093fb, #764ba2);
            color: white;
            border: none;
            padding: 8px 12px;
            border-radius: 50%;
            cursor: pointer;
            font-size: 16px;
            box-shadow: 0 4px 15px rgba(240, 147, 251, 0.3);
        }}
        .qty-btn:hover {{ transform: scale(1.1); }}
        .cart-total {{
            text-align: center;
            font-size: 22px;
            font-weight: 700;
            color: #ffffff;
            margin: 25px 0;
            background: rgba(255,255,255,0.1);
            backdrop-filter: blur(10px);
            padding: 20px;
            border-radius: 20px;
            box-shadow: 0 8px 32px rgba(0,0,0,0.1);
            text-shadow: 0 2px 4px rgba(0,0,0,0.3);
        }}
        .history-item h4 {{ font-size: 15px; margin-bottom: 8px; color: #ffffff; }}
        .history-item p {{ color: rgba(255,255,255,0.9); font-size: 13px; }}
        .history-logo {{
            text-align: center;
            margin-bottom: 30px;
        }}
        .history-logo img {{ width: 150px; height: 150px; border-radius: 50%; box-shadow: 0 8px 25px rgba(0,0,0,0.2); }}

        /* QR Modal */
        .qr-modal {{
            display: none;
            position: fixed;
            z-index: 1000;
            left: 0;
            top: 0;
            width: 100%;
            height: 100%;
            background-color: rgba(0,0,0,0.5);
            backdrop-filter: blur(10px);
        }}
        .qr-modal-content {{
            background: linear-gradient(135deg, #764ba2 0%, #f093fb 100%);
            margin: 10% auto;
            padding: 30px;
            border-radius: 25px;
            width: 80%;
            max-width: 400px;
            text-align: center;
            box-shadow: 0 20px 60px rgba(118, 75, 162, 0.5);
        }}
        .qr-modal-content img {{
            width: 200px;
            height: 200px;
            border-radius: 20px;
            margin-bottom: 20px;
        }}
        .close-qr {{
            color: #ffffff;
            float: right;
            font-size: 28px;
            font-weight: bold;
            cursor: pointer;
        }}
        .close-qr:hover {{
            color: #00f2fe;
        }}

        /* Responsive */
        @media (max-width: 480px) {{
            .home-grid {{ grid-template-columns: repeat(2, 1fr); gap: 15px; }}
            .results-grid {{ grid-template-columns: repeat(2, 1fr); gap: 15px; }}
            .menu-container {{ flex-direction: column; }}
            .menu-left {{ flex: none; width: 100%; max-height: 300px; /* Giới hạn chiều cao cho mobile */ }}
            .detail-panel {{ margin-left: 0; margin-top: 20px; width: 100%; }}
            .main-content {{ padding: 10px; margin-bottom: 90px; }}
            .logo {{ font-size: 26px; }}
            .logo i {{ font-size: 32px; }}
            .hero-title {{ font-size: 28px; }}
        }}
        .hidden {{ display: none; }}

        /* CẢI THIỆN GIAO DIỆN - PHẦN THÊM MỚI (KHÔNG XÓA CŨ) */

        /* Home Tab Improvements */
        .slideshow {{
            position: relative;
            width: 100%;
            height: 200px;
            overflow: hidden;
            border-radius: 25px;
            margin-bottom: 20px;
            box-shadow: 0 8px 32px rgba(0,0,0,0.1);
        }}
        .slide {{
            position: absolute;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            opacity: 0;
            transition: opacity 0.5s ease-in-out;
        }}
        .slide.active {{
            opacity: 1;
        }}
        .slide img {{
            width: 100%;
            height: 100%;
            object-fit: cover;
        }}
        .slide-caption {{
            position: absolute;
            bottom: 10px;
            left: 10px;
            background: rgba(0,0,0,0.5);
            padding: 5px 10px;
            border-radius: 10px;
            font-size: 14px;
        }}
        .home-search {{
            width: 100%;
            padding: 15px;
            margin-bottom: 20px;
            border-radius: 30px;
            border: 1px solid rgba(255,255,255,0.3);
            background: rgba(255,255,255,0.1);
            color: white;
            font-size: 16px;
            position: relative;
        }}
        .suggestions {{
            position: absolute;
            top: 100%;
            left: 0;
            width: 100%;
            background: rgba(255,255,255,0.1);
            border-radius: 0 0 20px 20px;
            display: none;
            max-height: 200px;
            overflow-y: auto;
        }}
        .suggestions div {{
            padding: 10px;
            cursor: pointer;
        }}
        .suggestions div:hover {{
            background: rgba(255,255,255,0.2);
        }}
        .custom-tray-btn {{
            background: linear-gradient(135deg, #f093fb, #764ba2);
            color: white;
            padding: 15px;
            border-radius: 30px;
            font-size: 16px;
            cursor: pointer;
            margin-top: 20px;
            text-align: center;
        }}

        /* Nhận diện Tab Improvements */
        .loading p {{
            font-size: 16px;
            animation: loadingText 3s infinite alternate;
        }}
        @keyframes loadingText {{
            0% {{ content: "Đang nếm thử món ăn..."; }}
            33% {{ content: "Đang xác định độ mặn..."; }}
            66% {{ content: "AI đang phân tích hương vị..."; }}
        }}
        .result-item .details {{
            font-size: 12px;
            color: #ccc;
            margin-top: 10px;
        }}
        .result-item .similar {{
            font-style: italic;
            color: #f093fb;
        }}
        .result-item select {{
            margin-top: 10px;
            padding: 5px;
            border-radius: 10px;
            background: rgba(255,255,255,0.1);
            color: white;
            border: 1px solid #00f2fe;
        }}

        /* Menu Tab Improvements */
        .menu-card:hover img {{
            animation: smoke 2s infinite;
        }}
        @keyframes smoke {{
            0% {{ transform: translateY(0); opacity: 1; }}
            100% {{ transform: translateY(-10px); opacity: 0; }}
        }}
        .filters {{
            display: flex;
            gap: 10px;
            margin-bottom: 20px;
        }}
        .filter-btn {{
            background: rgba(255,255,255,0.1);
            padding: 10px 15px;
            border-radius: 20px;
            cursor: pointer;
            transition: all 0.3s;
        }}
        .filter-btn.active {{
            background: #00f2fe;
            color: black;
        }}
        .filter-btn:hover {{
            background: #00f2fe;
            color: black;
        }}
        .menu-card .rating {{
            font-size: 12px;
            color: #ffd700;
            margin-top: 5px;
        }}
        .suggest-pair {{
            font-size: 12px;
            color: #f093fb;
            margin-top: 10px;
        }}

        /* Giỏ hàng Tab Improvements */
        .payment-methods {{
            display: flex;
            flex-direction: column;
            gap: 10px;
            margin-top: 20px;
        }}
        .payment-option {{
            background: rgba(255,255,255,0.1);
            padding: 10px;
            border-radius: 15px;
            cursor: pointer;
            transition: background 0.3s;
        }}
        .payment-option:hover {{
            background: rgba(255,255,255,0.2);
        }}
        .cart-suggestions {{
            margin-top: 20px;
            background: rgba(255,255,255,0.1);
            padding: 15px;
            border-radius: 20px;
        }}
        .cart-item {{
            animation: addToCart 0.5s ease;
        }}
        @keyframes addToCart {{
            0% {{ transform: scale(0.5); opacity: 0; }}
            100% {{ transform: scale(1); opacity: 1; }}
        }}

        /* Lịch sử Tab Improvements */
        .history-item img {{
            width: 50px;
            height: 50px;
            border-radius: 10px;
            margin-right: 10px;
        }}
        .history-item .reorder {{
            background: #00f2fe;
            color: black;
            padding: 5px 10px;
            border-radius: 10px;
            cursor: pointer;
            margin-left: 10px;
        }}
        .stats {{
            background: rgba(255,255,255,0.1);
            padding: 20px;
            border-radius: 20px;
            margin-bottom: 20px;
        }}

        /* CHỨC NĂNG MỚI - PHẦN THÊM MỚI (KHÔNG XÓA CŨ) */

        /* AI Chef Popup */
        .ai-chef-modal {{
            display: none;
            position: fixed;
            z-index: 1000;
            left: 0;
            top: 0;
            width: 100%;
            height: 100%;
            background-color: rgba(0,0,0,0.5);
        }}
        .ai-chef-content {{
            background: #fff;
            color: black;
            margin: 15% auto;
            padding: 20px;
            border-radius: 20px;
            width: 80%;
            max-width: 500px;
        }}
        .ai-chef-chat {{
            height: 200px;
            overflow-y: auto;
            margin-bottom: 10px;
        }}
        .ai-chef-input {{
            width: 100%;
            padding: 10px;
            margin-bottom: 10px;
        }}
        .ai-chef-send {{
            background: #4facfe;
            color: white;
            padding: 10px;
            border-radius: 10px;
            cursor: pointer;
        }}

        /* AR Khay Cơm Ảo */
        .ar-tray {{
            display: none;
            background: rgba(255,255,255,0.1);
            padding: 20px;
            border-radius: 20px;
            margin-top: 20px;
        }}
        .tray-grid {{
            display: grid;
            grid-template-columns: repeat(3, 1fr);
            gap: 10px;
            height: 200px;
            border: 2px dashed #00f2fe;
            position: relative;
        }}
        .tray-item {{
            background: rgba(0,242,254,0.2);
            height: 100px;
            border-radius: 10px;
            display: flex;
            align-items: center;
            justify-content: center;
            cursor: grab;
        }}

        /* Khám Phá Món Ăn Quanh Bạn */
        .explore-map {{
            background: rgba(255,255,255,0.1);
            padding: 20px;
            border-radius: 20px;
            margin-top: 20px;
        }}
        .mock-map {{
            height: 200px;
            background: url('https://via.placeholder.com/300x200?text=Map') no-repeat center;
            background-size: cover;
            border-radius: 10px;
        }}
        .regional-foods {{
            margin-top: 10px;
            list-style: none;
        }}

        /* Nhật Ký Vị Giác */
        .taste-journal {{
            background: rgba(255,255,255,0.1);
            padding: 20px;
            border-radius: 20px;
            margin-top: 20px;
        }}
        .timeline {{
            list-style: none;
        }}
        .timeline li {{
            margin-bottom: 10px;
            padding: 10px;
            background: rgba(0,242,254,0.1);
            border-radius: 10px;
        }}

        /* Bảng Xếp Hạng Món Hot */
        .hot-rankings {{
            background: rgba(255,255,255,0.1);
            padding: 20px;
            border-radius: 20px;
            margin-top: 20px;
        }}
        .ranking-list {{
            list-style: none;
        }}
        .ranking-item {{
            display: flex;
            align-items: center;
            margin-bottom: 10px;
        }}
        .ranking-item img {{
            width: 40px;
            height: 40px;
            border-radius: 50%;
            margin-right: 10px;
        }}

        /* Dark/Light Mode */
        body.light-mode {{
            background: linear-gradient(135deg, #f6d365 0%, #fda085 100%);
            color: #000;
        }}
        body.light-mode .header {{
            background: linear-gradient(135deg, #ff9966 0%, #ff5e62 100%);
        }}
        body.light-mode .tabs {{
            background: rgba(255, 182, 193, 0.9);
        }}
        body.light-mode .home-card, body.light-mode .result-item, body.light-mode .menu-card, body.light-mode .cart-item, body.light-mode .history-item {{
            background: rgba(255,255,255,0.8);
            color: #000;
        }}
        .theme-toggle {{
            position: absolute;
            top: 20px;
            right: 20px;
            cursor: pointer;
            font-size: 24px;
        }}

        /* So Sánh Món Modal */
        .compare-modal {{
            display: none;
            position: fixed;
            z-index: 1000;
            left: 0;
            top: 0;
            width: 100%;
            height: 100%;
            background-color: rgba(0,0,0,0.5);
        }}
        .compare-content {{
            background: #fff;
            color: black;
            margin: 15% auto;
            padding: 20px;
            border-radius: 20px;
            width: 80%;
            max-width: 600px;
        }}
        .compare-select {{
            display: flex;
            gap: 10px;
            margin-bottom: 20px;
        }}
        .compare-table {{
            width: 100%;
            border-collapse: collapse;
        }}
        .compare-table th, .compare-table td {{
            border: 1px solid #ddd;
            padding: 8px;
            text-align: left;
        }}

        /* Mini Profile */
        .profile {{
            position: absolute;
            top: 20px;
            left: 20px;
            display: flex;
            align-items: center;
            gap: 10px;
            cursor: pointer;
        }}
        .profile img {{
            width: 40px;
            height: 40px;
            border-radius: 50%;
        }}
        .badges {{
            display: flex;
            gap: 5px;
        }}
        .badge {{
            font-size: 12px;
            background: #ffd700;
            color: black;
            padding: 2px 5px;
            border-radius: 5px;
        }}

        /* Hiệu ứng thông minh */
        .tab-content {{
            animation: slideIn 0.5s ease;
        }}
        @keyframes slideIn {{
            from {{ transform: translateX(100%); }}
            to {{ transform: translateX(0); }}
        }}
        .menu-card img, .result-item img, .cart-item img {{
            transition: transform 0.3s;
        }}
        .menu-card:hover img, .result-item:hover img, .cart-item:hover img {{
            transform: scale(1.1) rotate(5deg);
        }}

        .new-tab-btn {{
            background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);
            color: white;
            padding: 10px 20px;
            border-radius: 20px;
            text-decoration: none;
            font-size: 14px;
            margin-left: 20px;
            box-shadow: 0 4px 15px rgba(79, 172, 254, 0.3);
            transition: all 0.3s;
        }}
        .new-tab-btn:hover {{
            transform: translateY(-2px);
            box-shadow: 0 8px 25px rgba(79, 172, 254, 0.4);
        }}

        /* Nút tròn góc trái cho chatbox FAQ */
        .chat-btn {{
            position: fixed;
            bottom: 100px; /* Không che tab dưới */
            left: 20px;
            width: 50px;
            height: 50px;
            background: linear-gradient(135deg, #4facfe, #00f2fe);
            border-radius: 50%;
            display: flex;
            align-items: center;
            justify-content: center;
            cursor: pointer;
            box-shadow: 0 4px 15px rgba(79, 172, 254, 0.3);
            z-index: 99; /* Không che các yếu tố khác */
            transition: all 0.3s;
        }}
        .chat-btn:hover {{
            transform: scale(1.1);
        }}
        .chat-btn i {{
            font-size: 24px;
            color: white;
        }}

        /* Chatbox Modal */
        .chat-modal {{
            display: none;
            position: fixed;
            z-index: 1000;
            left: 0;
            top: 0;
            width: 100%;
            height: 100%;
            background-color: rgba(0,0,0,0.5);
            backdrop-filter: blur(10px);
        }}
        .chat-modal-content {{
            background: linear-gradient(135deg, #764ba2 0%, #f093fb 100%);
            margin: 15% auto;
            padding: 20px;
            border-radius: 25px;
            width: 80%;
            max-width: 500px;
            max-height: 80vh;
            overflow-y: auto;
            text-align: left;
            box-shadow: 0 20px 60px rgba(118, 75, 162, 0.5);
        }}
        .chat-close {{
            color: #ffffff;
            float: right;
            font-size: 28px;
            font-weight: bold;
            cursor: pointer;
        }}
        .chat-close:hover {{
            color: #00f2fe;
        }}
        .faq-question {{
            cursor: pointer;
            font-weight: 600;
            margin-bottom: 10px;
            padding: 10px;
            background: rgba(255,255,255,0.1);
            border-radius: 10px;
        }}
        .faq-answer {{
            display: none;
            padding: 10px;
            background: rgba(255,255,255,0.05);
            border-radius: 10px;
            margin-bottom: 15px;
        }}

    </style>
</head>
<body>
    <div class="header">
        <div class="logo">
            <i class="fas fa-utensils"></i>
            FoodApp
            <button class="new-tab-btn" onclick="openNewTab()"><i class="fas fa-external-link-alt"></i> Mở trong tab mới</button>
        </div>
    </div>

    <div class="main-content">
        <!-- Home Tab - Với Hero Banner -->
        <div id="home" class="tab-content active">
            <div class="hero-banner">
                <h1 class="hero-title">Chào mừng đến với FoodApp!</h1>
                <p class="hero-subtitle">Khám phá thế giới ẩm thực huyền bí với AI nhận diện món ăn thông minh</p>
                <button class="hero-cta" onclick="showTab('menu')">
                    <i class="fas fa-utensils"></i> Đặt món ngay
                </button>
            </div>
            <div class="slideshow">
                <div class="slide active">
                    <img src="https://drive.google.com/uc?export=view&id=REPLACE_ID_HINH_BANNER_AN_NOI_BAT" alt="Món nổi bật">
                    <div class="slide-caption">Món nổi bật hôm nay: Thịt Kho</div>
                </div>
                <div class="slide">
                    <img src="https://drive.google.com/uc?export=view&id=REPLACE_ID_MON_GIAM_GIA" alt="Món giảm giá">
                    <div class="slide-caption">Giảm 20%: Canh Chua</div>
                </div>
                <div class="slide">
                    <img src="https://drive.google.com/uc?export=view&id=REPLACE_ID_GOI_Y_AI" alt="Gợi ý AI">
                    <div class="slide-caption">AI gợi ý: Rau Xào</div>
                </div>
            </div>
            <input type="text" class="home-search" placeholder="Tìm kiếm nhanh..." onkeyup="showSuggestions(this.value)">
            <div class="suggestions" id="homeSuggestions"></div>
            <div class="home-grid">
                <div class="home-card" onclick="showTab('recog')">
                    <img src="https://drive.google.com/uc?export=view&id=REPLACE_ID_ICON_QUET_KHAY" alt="Nhận diện">
                    <h3>Quét khay</h3>
                    <p>Chụp ảnh để tự động thêm món</p>
                </div>
                <div class="home-card" onclick="showTab('menu')">
                    <img src="https://drive.google.com/uc?export=view&id=REPLACE_ID_ICON_MENU" alt="Menu">
                    <h3>Duyệt menu</h3>
                    <p>Khám phá các món ngon</p>
                </div>
                <div class="home-card" onclick="showTab('cart')">
                    <img src="https://drive.google.com/uc?export=view&id=REPLACE_ID_ICON_GIO_HANG" alt="Giỏ">
                    <h3>Xem giỏ hàng</h3>
                    <p>Quản lý đơn hàng của bạn</p>
                </div>
            </div>
            <div class="custom-tray-btn" onclick="openCustomTray()">Tùy chỉnh khay cơm</div>
            <div style="text-align: center; color: rgba(255,255,255,0.8); font-size: 14px; padding: 30px; background: rgba(255,255,255,0.05); backdrop-filter: blur(10px); border-radius: 20px; box-shadow: 0 8px 32px rgba(0,0,0,0.1); border: 1px solid rgba(255,255,255,0.1); margin-top: 20px;">
                <i class="fas fa-star" style="color: #f093fb; margin-right: 10px;"></i>
                Mẹo: Sử dụng tab "Nhận diện" để quét khay cơm tự động tính tiền! Hỗ trợ camera và upload ảnh.
            </div>
        </div>

        <!-- Nhận diện Tab -->
        <div id="recog" class="tab-content">
            <div class="recog-section">
                <div class="upload-area" id="uploadArea">
                    <i class="fas fa-cloud-upload-alt" style="font-size: 56px; color: #00f2fe; margin-bottom: 20px; filter: drop-shadow(0 0 10px #00f2fe);"></i>
                    <p style="font-size: 18px; margin-bottom: 25px; font-weight: 500;">📤 Upload hoặc chụp ảnh khay cơm</p>
                    <div style="margin-bottom: 20px;">
                        <label style="margin-right: 20px;"><input type="radio" name="recog_mode" value="tray" checked> Nhận diện khay</label>
                        <label><input type="radio" name="recog_mode" value="single"> Nhận diện món đơn</label>
                    </div>
                    <input type="file" id="fileInput" accept="image/*" capture="environment">
                    <button class="upload-btn" onclick="document.getElementById('fileInput').click();"><i class="fas fa-folder-open"></i> Chọn file</button>
                    <button class="camera-btn" onclick="openCamera();"><i class="fas fa-camera"></i> Mở Camera</button>
                </div>

                <div class="guide-section">
                    <h3>Khám Phá Cuộc Phiêu Lưu Ẩm Thực!</h3>
                    <p>Bắt đầu hành trình bằng cách chụp ảnh khay cơm của bạn. Hãy tưởng tượng bạn là nhà thám hiểm khám phá những món ăn huyền bí, đẩy khung hình sao cho rõ nét để AI tiết lộ bí mật! (Thay ảnh minh họa dưới đây bằng ảnh của bạn sau)</p>
                    <img src="https://drive.google.com/uc?export=view&id=REPLACE_ID_HUONG_DAN_CHUP_ANH" alt="Hướng dẫn chụp ảnh">
                </div>

                <div class="camera-preview" id="cameraPreview">
                    <video id="cameraFeed" autoplay playsinline></video>
                    <canvas id="canvas" class="hidden"></canvas>
                    <div style="margin-top: 20px;">
                        <button class="upload-btn" onclick="capturePhoto();"><i class="fas fa-camera"></i> Chụp ảnh</button>
                        <button class="camera-btn" onclick="closeCamera();"><i class="fas fa-times"></i> Đóng</button>
                    </div>
                </div>

                <div class="loading" id="loading">
                    <div class="spinner"></div>
                    <p style="font-size: 16px; font-weight: 500;">Đang xử lý ảnh bằng AI huyền bí...</p>
                </div>

                <div id="resultsContainer" style="display: none;">
                    <h3 style="text-align: center; color: #ffffff; margin-bottom: 25px; font-size: 22px; font-weight: 600; text-shadow: 0 2px 4px rgba(0,0,0,0.5);">
                        <i class="fas fa-utensils" style="color: #f093fb;"></i> Kết quả nhận diện
                    </h3>
                    <div class="results-grid" id="resultsGrid"></div>

                    <div class="summary-box">
                        <div style="font-size: 16px;">Tổng cộng:</div>
                        <div class="summary-total" id="totalAmount">0 VNĐ</div>
                        <button class="checkout-btn" onclick="proceedToCheckout();">
                            <i class="fas fa-credit-card"></i> Thanh toán
                        </button>
                    </div>

                    <div class="qr-section" id="qrSection">
                        <h3 style="color: #ffffff; margin-bottom: 20px; font-weight: 600; text-shadow: 0 2px 4px rgba(0,0,0,0.5);">
                            <i class="fas fa-qrcode" style="color: #00f2fe;"></i> Quét mã QR thanh toán
                        </h3>
                        <img id="qrCode" src="" alt="QR Code">
                        <p style="margin-top: 20px; font-size: 16px;">
                            Số tiền: <strong id="qrAmount">0 VNĐ</strong>
                        </p>
                        <button class="upload-btn" onclick="completePayment('recog');" style="margin-top: 20px; width: auto; padding: 15px 30px;">
                            <i class="fas fa-check"></i> Hoàn tất
                        </button>
                    </div>
                </div>
            </div>
        </div>

        <!-- Menu Tab -->
        <div id="menu" class="tab-content">
            <input type="text" class="menu-search" placeholder="Tìm món ăn yêu thích..." onkeyup="searchMenu(this.value);">
            <div class="filters">
                <div class="filter-btn active" onclick="filterMenu('all')">Tất cả</div>
                <div class="filter-btn" onclick="filterMenu('low-cal')">Ít calo</div>
                <div class="filter-btn" onclick="filterMenu('veg')">Chay</div>
                <div class="filter-btn" onclick="filterMenu('high-price')">Giá cao</div>
            </div>
            <div class="menu-container">
                <div class="menu-left">
                    <!-- Menu cards with added data for filtering -->
                    <div class="menu-card" onclick="showDetail(this)" data-name="Cơm Trắng" data-price="10000" data-calories="240" data-category="non-veg" data-prep-time="20 phút" data-difficulty="Dễ" data-nutrition="Protein: 4g, Carb: 53g, Fat: 0.6g" data-origin="Việt Nam" data-pairing="Kết hợp với các món mặn" data-story="Lúa gạo trắng là biểu tượng của biểu tượng của nền văn minh lúa nước Việt Nam. Nó bắt nguồn từ khoảng 2000 TCN. Kỹ thuật canh tác ướt của văn hóa Đông Sơn. Nuôi dưỡng hàng ngàn năm lịch sử ẩm thực dân tộc. Món ăn đơn giản nhưng thiết yếu trong bữa cơm hàng ngày. Mang ý nghĩa văn hóa sâu sắc. Thể hiện sự cần cù lao động của người nông dân.">
                        <img src="https://drive.google.com/uc?export=view&id=REPLACE_ID_COM" alt="Cơm Trắng">
                        <div class="menu-card-info">
                            <h3>Cơm Trắng</h3>
                            <div class="price">10,000 VNĐ</div>
                            <div class="rating">★★★★☆ (4.5)</div>
                        </div>
                        <div class="suggest-pair">Gợi ý kèm: Canh Rau</div>
                    </div>
                    <div class="menu-card" onclick="showDetail(this)" data-name="Đậu Hũ Sốt Cà" data-price="25000" data-calories="180" data-category="veg" data-prep-time="30 phút" data-difficulty="Trung bình" data-nutrition="Protein: 12g, Carb: 10g, Fat: 8g" data-origin="Sapa, Việt Nam" data-pairing="Ăn kèm cơm trắng" data-story="Món ăn truyền thống từ vùng núi Sapa. Với đậu phụ giòn rụm ngấm sốt cà chua cay nồng. Phản ánh sự kết hợp hài hòa giữa nguyên liệu địa phương. Kỹ thuật chiên xào đơn giản của ẩm thực miền núi Việt. Món chay phổ biến trong các bữa ăn gia đình. Được yêu thích bởi vị chua ngọt hấp dẫn. Mang hương vị núi rừng Tây Bắc.">
                        <img src="https://drive.google.com/uc?export=view&id=REPLACE_ID_DAU_HU_SOT_CA" alt="Đậu Hũ Sốt Cà">
                        <div class="menu-card-info">
                            <h3>Đậu Hũ Sốt Cà</h3>
                            <div class="price">25,000 VNĐ</div>
                            <div class="rating">★★★★★ (5.0)</div>
                        </div>
                        <div class="suggest-pair">Gợi ý kèm: Cơm Trắng</div>
                    </div>
                    <div class="menu-card" onclick="showDetail(this)" data-name="Cá Hũ Kho" data-price="30000" data-calories="320" data-category="non-veg" data-prep-time="60 phút" data-difficulty="Trung bình" data-nutrition="Protein: 25g, Carb: 5g, Fat: 22g" data-origin="Miền Nam Việt Nam" data-pairing="Ăn với cơm trắng và rau sống" data-story="Kỹ thuật 'kho' cổ truyền từ đồng bằng sông Cửu Long. Cá kho tộ là món ăn dân dã. Caramel hóa ngọt mặn. Tượng trưng cho sự kiên nhẫn và tiết kiệm trong nấu nướng Việt Nam. Qua hàng thế kỷ. Món ăn phổ biến trong bữa cơm gia đình miền Nam. Mang hương vị sông nước. Thường nấu trong nồi đất để tăng độ ngon.">
                        <img src="https://drive.google.com/uc?export=view&id=REPLACE_ID_CA_HU_KHO" alt="Cá Hũ Kho">
                        <div class="menu-card-info">
                            <h3>Cá Hũ Kho</h3>
                            <div class="price">30,000 VNĐ</div>
                            <div class="rating">★★★★☆ (4.2)</div>
                        </div>
                        <div class="suggest-pair">Gợi ý kèm: Rau Xào</div>
                    </div>
                    <div class="menu-card" onclick="showDetail(this)" data-name="Thịt Kho" data-price="30000" data-calories="530" data-category="non-veg" data-prep-time="90 phút" data-difficulty="Trung bình" data-nutrition="Protein: 30g, Carb: 10g, Fat: 40g" data-origin="Miền Nam Việt Nam" data-pairing="Ăn với cơm trắng và dưa chua" data-story="Thịt kho tàu có nguồn gốc ảnh hưởng từ ẩm thực Trung Hoa. Nhưng được Việt hóa. Là món 'linh hồn' của Tết Nguyên Đán. Đại diện cho sự thịnh vượng và đoàn tụ gia đình miền Nam. Món ăn đậm đà hương vị. Thường ăn với cơm trắng nóng hổi. Được nấu chậm để thịt mềm tan. Mang ý nghĩa may mắn cho năm mới.">
                        <img src="https://drive.google.com/uc?export=view&id=REPLACE_ID_THIT_KHO" alt="Thịt Kho">
                        <div class="menu-card-info">
                            <h3>Thịt Kho</h3>
                            <div class="price">30,000 VNĐ</div>
                            <div class="rating">★★★★★ (4.8)</div>
                        </div>
                        <div class="suggest-pair">Gợi ý kèm: Canh Chua</div>
                    </div>
                    <div class="menu-card" onclick="showDetail(this)" data-name="Thịt Kho 1 Trứng" data-price="36000" data-calories="530" data-category="non-veg" data-prep-time="90 phút" data-difficulty="Trung bình" data-nutrition="Protein: 35g, Carb: 10g, Fat: 40g" data-origin="Việt Nam" data-pairing="Ăn với cơm trắng" data-story="Biến tấu của thịt kho với 1 quả trứng. Là 'món ăn quốc hồn quốc túy' Việt Nam. Thường chế biến Tết để cầu mong may mắn. Thể hiện sự khéo léo trong việc kết hợp protein từ thịt và trứng. Món ăn giàu dinh dưỡng. Phổ biến trong các dịp lễ hội. Mang màu sắc vàng đẹp mắt. Tượng trưng cho sự sung túc.">
                        <img src="https://drive.google.com/uc?export=view&id=REPLACE_ID_THIT_KHO_1_TRUNG" alt="Thịt Kho 1 Trứng">
                        <div class="menu-card-info">
                            <h3>Thịt Kho 1 Trứng</h3>
                            <div class="price">36,000 VNĐ</div>
                            <div class="rating">★★★★☆ (4.3)</div>
                        </div>
                        <div class="suggest-pair">Gợi ý kèm: Canh Rau</div>
                    </div>
                    <div class="menu-card" onclick="showDetail(this)" data-name="Thịt Kho 2 Trứng" data-price="42000" data-calories="580" data-category="non-veg" data-prep-time="90 phút" data-difficulty="Trung bình" data-nutrition="Protein: 40g, Carb: 10g, Fat: 42g" data-origin="Việt Nam" data-pairing="Ăn với cơm trắng" data-story="Biến tấu của thịt kho với 2 quả trứng. Là 'món ăn quốc hồn quốc túy' Việt Nam. Thường chế biến Tết để cầu mong may mắn. Thể hiện sự khéo léo trong việc kết hợp protein từ thịt và trứng. Món ăn giàu dinh dưỡng. Phổ biến trong các dịp lễ hội. Mang màu sắc vàng đẹp mắt. Tượng trưng cho sự sung túc.">
                        <img src="https://drive.google.com/uc?export=view&id=REPLACE_ID_THIT_KHO_2_TRUNG" alt="Thịt Kho 2 Trứng">
                        <div class="menu-card-info">
                            <h3>Thịt Kho 2 Trứng</h3>
                            <div class="price">42,000 VNĐ</div>
                            <div class="rating">★★★★★ (4.7)</div>
                        </div>
                        <div class="suggest-pair">Gợi ý kèm: Rau Xào</div>
                    </div>
                    <div class="menu-card" onclick="showDetail(this)" data-name="Canh Chua Có Cà" data-price="25000" data-calories="240" data-category="non-veg" data-prep-time="40 phút" data-difficulty="Trung bình" data-nutrition="Protein: 20g, Carb: 15g, Fat: 12g" data-origin="Miền Nam Việt Nam" data-pairing="Ăn với cơm trắng" data-story="Bản địa từ đồng bằng sông Cửu Long. Canh chua với cà (dứa) tạo vị chua ngọt đặc trưng từ me và hải sản sông nước. Là biểu tượng của sự phong phú thiên nhiên miền Nam Việt Nam. Món ăn thanh mát. Thường ăn kèm với với cá hoặc tôm. Giải nhiệt tốt trong ngày hè. Được yêu thích bởi vị chua thanh quyến rũ.">
                        <img src="https://drive.google.com/uc?export=view&id=REPLACE_ID_CANH_CHUA_CO_CA" alt="Canh Chua Có Cà">
                        <div class="menu-card-info">
                            <h3>Canh Chua Có Cà</h3>
                            <div class="price">25,000 VNĐ</div>
                            <div class="rating">★★★★☆ (4.6)</div>
                        </div>
                        <div class="suggest-pair">Gợi ý kèm: Thịt Kho</div>
                    </div>
                    <div class="menu-card" onclick="showDetail(this)" data-name="Canh Chua Không Cà" data-price="10000" data-calories="176" data-category="non-veg" data-prep-time="30 phút" data-difficulty="Dễ" data-nutrition="Protein: 18g, Carb: 12g, Fat: 8g" data-origin="Miền Nam Việt Nam" data-pairing="Ăn với cơm trắng" data-story="Phiên bản đơn giản của canh chua miền Nam. Không dứa nhưng vẫn giữ vị chua từ me. Phản ánh lối sống đơn giản. Sử dụng nguyên liệu địa phương dồi dào ở vùng sông nước Việt. Món ăn dễ làm. Phù hợp cho bữa cơm hàng ngày. Thanh đạm và bổ dưỡng.">
                        <img src="https://drive.google.com/uc?export=view&id=REPLACE_ID_CANH_CHUA_KHONG_CA" alt="Canh Chua Không Cà">
                        <div class="menu-card-info">
                            <h3>Canh Chua Không Cà</h3>
                            <div class="price">10,000 VNĐ</div>
                            <div class="rating">★★★☆☆ (3.8)</div>
                        </div>
                        <div class="suggest-pair">Gợi ý kèm: Sườn Nướng</div>
                    </div>
                    <div class="menu-card" onclick="showDetail(this)" data-name="Sườn Nướng" data-price="30000" data-calories="373" data-category="non-veg" data-prep-time="45 phút" data-difficulty="Trung bình" data-nutrition="Protein: 28g, Carb: 15g, Fat: 25g" data-origin="Sài Gòn, Việt Nam" data-pairing="Ăn với salad" data-story="Sườn nướng đường phố Sài Gòn. Ướp sả mật ong nướng than. Bắt nguồn từ văn hóa ăn vặt miền Nam. Là món ăn nhanh tiện lợi. Gắn liền với nhịp sống sôi động của đô thị Việt. Mùi thơm nức mũi. Gường ăn kèm salad. Gược ưa chuộng trong các buổi tiệc nướng.">
                        <img src="https://drive.google.com/uc?export=view&id=REPLACE_ID_SUON_NUONG" alt="Sườn Nướng">
                        <div class="menu-card-info">
                            <h3>Sườn Nướng</h3>
                            <div class="price">30,000 VNĐ</div>
                            <div class="rating">★★★★★ (4.9)</div>
                        </div>
                        <div class="suggest-pair">Gợi ý kèm: Canh Chua Không Cà</div>
                    </div>
                    <div class="menu-card" onclick="showDetail(this)" data-name="Canh Rau" data-price="7000" data-calories="150" data-category="veg" data-prep-time="25 phút" data-difficulty="Dễ" data-nutrition="Protein: 10g, Carb: 8g, Fat: 5g" data-origin="Việt Nam" data-pairing="Ăn với các món mặn" data-story="Canh rau đơn giản từ truyền thống nông nghiệp Việt. Sử dụng rau địa phương như rau muống. Thể hiện triết lý 'rau sạch từ vườn nhà'. Sự cân bằng dinh dưỡng trong bữa ăn hàng ngày. Món ăn thanh đạm. Giàu vitamin. Giúp tiêu hóa tốt hơn.">
                        <img src="https://drive.google.com/uc?export=view&id=REPLACE_ID_CANH_RAU" alt="Canh Rau">
                        <div class="menu-card-info">
                            <h3>Canh Rau</h3>
                            <div class="price">7,000 VNĐ</div>
                            <div class="rating">★★★☆☆ (3.5)</div>
                        </div>
                        <div class="suggest-pair">Gợi ý kèm: Trứng Chiên</div>
                    </div>
                    <div class="menu-card" onclick="showDetail(this)" data-name="Rau Xào" data-price="10000" data-calories="114" data-category="veg" data-prep-time="15 phút" data-difficulty="Dễ" data-nutrition="Protein: 5g, Carb: 15g, Fat: 4g" data-origin="Việt Nam" data-pairing="Ăn với thịt hoặc cá" data-story="Rau xào thập cẩm là món phụ kinh điển trong ẩm thực gia đình Việt. Bắt nguồn từ kỹ thuật xào nhanh giữ độ giòn. Nhấn mạnh sự tươi ngon của rau củ theo mùa ở đồng quê. Món ăn lành mạnh. Giàu chất xơ. Dễ kết hợp với các món chính. Giúp cân bằng bữa ăn.">
                        <img src="https://drive.google.com/uc?export=view&id=REPLACE_ID_RAU_XAO" alt="Rau Xào">
                        <div class="menu-card-info">
                            <h3>Rau Xào</h3>
                            <div class="price">10,000 VNĐ</div>
                            <div class="rating">★★★★☆ (4.0)</div>
                        </div>
                        <div class="suggest-pair">Gợi ý kèm: Cá Hũ Kho</div>
                    </div>
                    <div class="menu-card" onclick="showDetail(this)" data-name="Trứng Chiên" data-price="25000" data-calories="250" data-category="non-veg" data-prep-time="10 phút" data-difficulty="Dễ" data-nutrition="Protein: 20g, Carb: 2g, Fat: 18g" data-origin="Việt Nam" data-pairing="Ăn với cơm trắng" data-story="Trứng chiên (trứng chiên thịt bằm) là món ăn sáng hoặc phụ đơn giản. Ảnh hưởng từ ẩm thực Pháp-Việt. Tượng trưng cho sự tiện lợi và dinh dưỡng cao trong bữa ăn hàng ngày của người Việt. Món ăn nhanh chóng. Giàu protein. Phổ biến trong các quán ăn đường phố. Dễ biến tấu với các nguyên liệu khác.">
                        <img src="https://drive.google.com/uc?export=view&id=REPLACE_ID_TRUNG_CHIEN" alt="Trứng Chiên">
                        <div class="menu-card-info">
                            <h3>Trứng Chiên</h3>
                            <div class="price">25,000 VNĐ</div>
                            <div class="rating">★★★★☆ (4.1)</div>
                        </div>
                        <div class="suggest-pair">Gợi ý kèm: Cơm Trắng</div>
                    </div>
                </div>
                <div class="detail-panel" id="detailPanel">
                    <img id="detailImage" src="" alt="">
                    <table class="detail-table">
                        <tr>
                            <th>Tên món</th>
                            <td id="detailName"></td>
                        </tr>
                        <tr>
                            <th>Giá</th>
                            <td id="detailPrice"></td>
                        </tr>
                        <tr>
                            <th>Calo</th>
                            <td id="detailCalories"></td>
                        </tr>
                        <tr>
                            <th>Số lượng</th>
                            <td id="detailQuantity"></td>
                        </tr>
                        <tr>
                            <th>Nguyên liệu</th>
                            <td id="detailIngredients"></td>
                        </tr>
                        <tr>
                            <th>Thời gian chuẩn bị</th>
                            <td id="detailPrepTime"></td>
                        </tr>
                        <tr>
                            <th>Độ khó</th>
                            <td id="detailDifficulty"></td>
                        </tr>
                        <tr>
                            <th>Thông tin dinh dưỡng</th>
                            <td id="detailNutrition"></td>
                        </tr>
                        <tr>
                            <th>Nguồn gốc</th>
                            <td id="detailOrigin"></td>
                        </tr>
                        <tr>
                            <th>Kết hợp với</th>
                            <td id="detailPairing"></td>
                        </tr>
                    </table>
                    <div class="detail-story" id="detailStory"></div>
                    <button class="detail-add-btn" id="detailAddBtn">Thêm vào giỏ</button>
                </div>
            </div>
        </div>

        <!-- Giỏ hàng Tab -->
        <div id="cart" class="tab-content">
            <ul class="cart-list" id="cartItems"></ul>
            <div class="cart-total" id="cartTotal">Tổng: 0 VNĐ</div>
            <div class="payment-methods">
                <div class="payment-option" onclick="handlePayment('qr')">QR Code</div>
                <div class="payment-option" onclick="handlePayment('credit')">Thẻ tín dụng</div>
                <div class="payment-option" onclick="handlePayment('cash')">Tiền mặt</div>
            </div>
            <div class="cart-suggestions">
                <h4>Gợi ý thêm:</h4>
                <p>Dựa trên giỏ hàng, thử thêm Canh Rau?</p>
            </div>
            <button class="checkout-btn" onclick="checkout();" style="margin-top: 25px;">
                <i class="fas fa-shopping-cart"></i> Thanh toán
            </button>
        </div>

        <!-- Lịch sử Tab -->
        <div id="history" class="tab-content">
            <div class="stats">
                <h4>Thống kê ăn uống</h4>
                <p>Bạn ăn 3 lần thịt kho trong tuần qua.</p>
            </div>
            <ul class="history-list" id="historyItems"></ul>
        </div>
    </div>

    <!-- Bottom Navigation -->
    <div class="tabs">
        <button class="tab-btn active" onclick="showTab('home');">
            <i class="fas fa-home"></i>
            <span>Home</span>
        </button>
        <button class="tab-btn" onclick="showTab('recog');">
            <i class="fas fa-camera"></i>
            <span>Nhận diện</span>
        </button>
        <button class="tab-btn" onclick="showTab('menu');">
            <i class="fas fa-utensils"></i>
            <span>Menu</span>
        </button>
        <button class="tab-btn" onclick="showTab('cart');">
            <i class="fas fa-shopping-bag"></i>
            <span>Giỏ hàng</span>
        </button>
        <button class="tab-btn" onclick="showTab('history');">
            <i class="fas fa-history"></i>
            <span>Lịch sử</span>
        </button>
    </div>

    <!-- QR Modal -->
    <div id="qrModal" class="qr-modal">
        <div class="qr-modal-content">
            <span class="close-qr" onclick="closeQRModal()">&times;</span>
            <h3><i class="fas fa-qrcode" style="color: #00f2fe;"></i> Quét mã QR để chuyển khoản</h3>
            <img id="modalQRCode" src="" alt="QR Code">
            <p>Số tiền: <strong id="modalQRAmount">0 VNĐ</strong></p>
            <p>Chuyển khoản đến tài khoản FoodApp</p>
            <button class="upload-btn" onclick="completePayment('menu');" style="margin-top: 20px; width: auto; padding: 15px 30px;">
                <i class="fas fa-check"></i> Hoàn tất
            </button>
        </div>
    </div>

    <!-- AI Chef Modal -->
    <div id="aiChefModal" class="ai-chef-modal">
        <div class="ai-chef-content">
            <span onclick="closeAIChef()" style="float: right; cursor: pointer;">&times;</span>
            <h3>AI Chef</h3>
            <div class="ai-chef-chat" id="aiChat"></div>
            <input type="text" class="ai-chef-input" id="aiInput" placeholder="Hỏi tôi về món ăn...">
            <button class="ai-chef-send" onclick="sendAIQuestion()">Gửi</button>
        </div>
    </div>

    <!-- AR Khay Cơm Ảo -->
    <div id="arTray" class="ar-tray">
        <h3>AR Khay Cơm Ảo</h3>
        <div class="tray-grid" id="trayGrid">
            <div class="tray-item" draggable="true">Ô 1</div>
            <div class="tray-item" draggable="true">Ô 2</div>
            <div class="tray-item" draggable="true">Ô 3</div>
            <div class="tray-item" draggable="true">Ô 4</div>
            <div class="tray-item" draggable="true">Ô 5</div>
        </div>
        <p>Tổng calo: <span id="trayCalo">0</span></p>
    </div>

    <!-- Khám Phá Món Ăn Quanh Bạn -->
    <div class="explore-map">
        <h3>Khám phá món ăn quanh bạn</h3>
        <div class="mock-map"></div>
        <ul class="regional-foods">
            <li>Miền Bắc: Phở</li>
            <li>Miền Nam: Bánh Mì</li>
        </ul>
    </div>

    <!-- Nhật Ký Vị Giác -->
    <div class="taste-journal">
        <h3>Nhật ký vị giác</h3>
        <ul class="timeline" id="tasteTimeline"></ul>
        <input type="text" id="tasteInput" placeholder="Ghi cảm nhận...">
        <button onclick="addTasteNote()">Thêm</button>
    </div>

    <!-- Bảng Xếp Hạng Món Hot -->
    <div class="hot-rankings">
        <h3>Top 5 món hot</h3>
        <ul class="ranking-list">
            <li class="ranking-item"><img src="https://drive.google.com/uc?export=view&id=REPLACE_ID_THIT_KHO" alt="">1. Thịt Kho</li>
            <li class="ranking-item"><img src="https://drive.google.com/uc?export=view&id=REPLACE_ID_CANH_CHUA_CO_CA" alt="">2. Canh Chua</li>
        </ul>
    </div>

    <!-- So Sánh Món Modal -->
    <div id="compareModal" class="compare-modal">
        <div class="compare-content">
            <span onclick="closeCompare()" style="float: right; cursor: pointer;">&times;</span>
            <h3>So sánh món</h3>
            <div class="compare-select">
                <select id="dish1">
                    <option>Cơm Trắng</option>
                    <option>Đậu Hũ Sốt Cà</option>
                </select>
                <select id="dish2">
                    <option>Đậu Hũ Sốt Cà</option>
                    <option>Cơm Trắng</option>
                </select>
            </div>
            <table class="compare-table">
                <tr><th>Tiêu chí</th><th>Món 1</th><th>Món 2</th></tr>
                <tr><td>Calo</td><td>240</td><td>180</td></tr>
                <tr><td>Giá</td><td>10000</td><td>25000</td></tr>
            </table>
            <button onclick="compareDishes()">So sánh</button>
        </div>
    </div>

    <!-- Mini Profile -->
    <div class="profile">
        <img src="https://drive.google.com/uc?export=view&id=REPLACE_ID_GIAM_GIA" alt="Avatar">
        <span>User</span>
        <div class="badges">
            <div class="badge">Food Explorer</div>
            <div class="badge">Healthy Eater</div>
        </div>
    </div>

    <!-- Theme Toggle -->
    <i class="fas fa-moon theme-toggle" onclick="toggleTheme()"></i>

    <!-- Nút tròn chatbox -->
    <div class="chat-btn" onclick="openChatModal()">
        <i class="fas fa-question-circle"></i>
    </div>

    <!-- Chatbox Modal -->
    <div id="chatModal" class="chat-modal">
        <div class="chat-modal-content">
            <span class="chat-close" onclick="closeChatModal()">&times;</span>
            <h3>Hỏi đáp thường gặp</h3>
            <div class="faq-question" onclick="toggleAnswer(0)">Làm thế nào để quét khay cơm?</div>
            <div class="faq-answer">Sử dụng tab "Nhận diện", chọn upload ảnh hoặc mở camera để chụp khay cơm. AI sẽ tự động phân tích và tính tiền.</div>
            <div class="faq-question" onclick="toggleAnswer(1)">Tôi có thể chỉnh sửa nhận diện không?</div>
            <div class="faq-answer">Có, sau khi nhận diện, bạn có thể chọn món từ dropdown để chỉnh sửa thủ công.</div>
            <div class="faq-question" onclick="toggleAnswer(2)">Làm thế nào để thanh toán?</div>
            <div class="faq-answer">Sau khi thêm vào giỏ, chọn phương thức thanh toán như QR, thẻ tín dụng hoặc tiền mặt.</div>
            <div class="faq-question" onclick="toggleAnswer(3)">Menu có lọc theo loại không?</div>
            <div class="faq-answer">Có, sử dụng bộ lọc ở tab Menu để lọc theo calo thấp, chay, giá cao, v.v.</div>
            <div class="faq-question" onclick="toggleAnswer(4)">Lịch sử đơn hàng ở đâu?</div>
            <div class="faq-answer">Trong tab "Lịch sử", bạn có thể xem các đơn hàng cũ và reorder nếu muốn.</div>
        </div>
    </div>

    <script>
        const API_URL = '{public_url}';
        let stream = null;
        let cart = JSON.parse(localStorage.getItem('cart')) || [];
        let currentRecognitionResults = null;
        let lastSource = '';

        const foodImages = {{
            'Cơm Trắng': 'https://drive.google.com/uc?export=view&id=REPLACE_ID_COM',
            'Đậu Hũ Sốt Cà': 'https://drive.google.com/uc?export=view&id=REPLACE_ID_DAU_HU_SOT_CA',
            'Cá Hũ Kho': 'https://drive.google.com/uc?export=view&id=REPLACE_ID_CA_HU_KHO',
            'Thịt Kho': 'https://drive.google.com/uc?export=view&id=REPLACE_ID_THIT_KHO',
            'Thịt Kho 1 Trứng': 'https://drive.google.com/uc?export=view&id=REPLACE_ID_THIT_KHO_1_TRUNG',
            'Thịt Kho 2 Trứng': 'https://drive.google.com/uc?export=view&id=REPLACE_ID_THIT_KHO_2_TRUNG',
            'Canh Chua Có Cà': 'https://drive.google.com/uc?export=view&id=REPLACE_ID_CANH_CHUA_CO_CA',
            'Canh Chua Không Cà': 'https://drive.google.com/uc?export=view&id=REPLACE_ID_CANH_CHUA_KHONG_CA',
            'Sườn Nướng': 'https://drive.google.com/uc?export=view&id=REPLACE_ID_SUON_NUONG',
            'Canh Rau': 'https://drive.google.com/uc?export=view&id=REPLACE_ID_CANH_RAU',
            'Rau Xào': 'https://drive.google.com/uc?export=view&id=REPLACE_ID_RAU_XAO',
            'Trứng Chiên': 'https://drive.google.com/uc?export=view&id=REPLACE_ID_TRUNG_CHIEN'
        }};

        // ============= TAB NAVIGATION =============
        function showTab(tabName) {{
            document.querySelectorAll('.tab-content').forEach(tab => tab.classList.remove('active'));
            document.querySelectorAll('.tab-btn').forEach(btn => btn.classList.remove('active'));
            document.getElementById(tabName).classList.add('active');
            event.currentTarget.classList.add('active');
            if (tabName === 'history') loadHistory();
            if (tabName === 'cart') updateCart();
        }}

        // File input change event - Sửa để hỗ trợ capture environment cho mobile
        document.getElementById('fileInput').addEventListener('change', (e) => {{
            const file = e.target.files[0];
            if (file) {{
                lastSource = 'recog';
                processImage(file);
            }}
        }});

        function openCamera() {{
            const cameraPreview = document.getElementById('cameraPreview');
            const video = document.getElementById('cameraFeed');
            const constraints = {{
                video: {{
                    facingMode: 'environment'
                }}
            }};
            navigator.mediaDevices.getUserMedia(constraints)
            .then(s => {{
                stream = s;
                video.srcObject = stream;
                cameraPreview.classList.add('active');
                document.getElementById('uploadArea').style.display = 'none';
            }})
            .catch(err => {{
                // Fallback nếu không hỗ trợ environment
                const fallbackConstraints = {{ video: true }};
                navigator.mediaDevices.getUserMedia(fallbackConstraints)
                .then(s => {{
                    stream = s;
                    video.srcObject = stream;
                    cameraPreview.classList.add('active');
                    document.getElementById('uploadArea').style.display = 'none';
                }})
                .catch(fallbackErr => alert('⚠️ Không thể truy cập camera: ' + fallbackErr.message));
            }});
        }}

        function capturePhoto() {{
            const video = document.getElementById('cameraFeed');
            const canvas = document.getElementById('canvas');
            const context = canvas.getContext('2d');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            context.drawImage(video, 0, 0);
            canvas.toBlob(blob => {{
                lastSource = 'recog';
                closeCamera();
                processImage(blob, 'captured.jpg');
            }}, 'image/jpeg', 0.8);  // Chất lượng 80% để test nhanh
        }}

        function closeCamera() {{
            if (stream) {{
                stream.getTracks().forEach(track => track.stop());
                stream = null;
            }}
            document.getElementById('cameraPreview').classList.remove('active');
            document.getElementById('uploadArea').style.display = 'block';
        }}

        async function processImage(imageFile, filename = null) {{
            const mode = document.querySelector('input[name="recog_mode"]:checked').value;
            const endpoint = mode === 'tray' ? '/detect_tray' : '/detect_single';
            document.getElementById('uploadArea').style.display = 'none';
            document.getElementById('cameraPreview').style.display = 'none';
            document.getElementById('resultsContainer').style.display = 'none';
            document.getElementById('loading').classList.add('active');
            try {{
                const formData = new FormData();
                formData.append('file', imageFile, filename || imageFile.name || 'image.jpg');
                const response = await fetch(`${{API_URL}}${{endpoint}}`, {{
                    method: 'POST',
                    body: formData,
                    headers: {{
                        'ngrok-skip-browser-warning': 'true'
                    }}
                }});
                if (!response.ok) throw new Error('Lỗi từ server');
                const data = await response.json();
                if (data.success) {{
                    currentRecognitionResults = data;
                    displayRecognitionResults(data);
                }} else {{
                    alert('❌ ' + (data.error || 'Lỗi không xác định'));
                    resetRecognition();
                }}
            }} catch (error) {{
                console.error('Error:', error);
                alert('❌ Lỗi kết nối: ' + error.message + '. Hãy thử lại hoặc kiểm tra server.');
                resetRecognition();
            }}
        }}

        function displayRecognitionResults(data) {{
            document.getElementById('loading').classList.remove('active');
            document.getElementById('resultsContainer').style.display = 'block';
            const grid = document.getElementById('resultsGrid');
            grid.innerHTML = '';
            data.results.forEach(item => {{
                const div = document.createElement('div');
                div.className = 'result-item';
                div.innerHTML = `
                    <img src="${{item.image}}" alt="${{item.food_name}}">
                    <div class="result-label">${{item.food_name}}</div>
                    <div class="result-conf">${{item.confidence}}% chính xác</div>
                    <div class="result-price">${{formatPrice(item.price)}}</div>
                    <div class="details">Tỷ lệ nhận dạng: ${{item.confidence}}%</div>
                    <div class="similar">Món tương tự: Cơm Trắng</div>
                    <select onchange="editFoodName(this, ${{item.id}})">
                        <option>${{item.food_name}}</option>
                        <option>Cơm Trắng</option>
                        <option>Thịt Kho</option>
                    </select>
                `;
                grid.appendChild(div);
            }});
            document.getElementById('totalAmount').textContent = formatPrice(data.total_price);
        }}

        function editFoodName(select, id) {{
            // Logic chỉnh sửa thủ công (mock)
            alert(`Đổi món thành: ${{select.value}}`);
        }}

        function proceedToCheckout() {{
            if (!currentRecognitionResults) return;
            currentRecognitionResults.results.forEach(item => {{
                cart.push({{
                    name: item.food_name,
                    price: item.price,
                    qty: 1,
                    source: 'Nhận diện khay'
                }});
            }});
            updateCart();
            showQR(currentRecognitionResults.total_price, false);
            saveToHistory(currentRecognitionResults.results, currentRecognitionResults.total_price);
            alert('✅ Đã thêm ' + currentRecognitionResults.results.length + ' món vào giỏ hàng!');
        }}

        function resetRecognition() {{
            document.getElementById('uploadArea').style.display = 'block';
            document.getElementById('resultsContainer').style.display = 'none';
            document.getElementById('qrSection').classList.remove('active');
            document.getElementById('fileInput').value = '';
            currentRecognitionResults = null;
            closeCamera();
        }}

        function searchMenu(query) {{
            document.querySelectorAll('.menu-card').forEach(card => {{
                const text = card.textContent.toLowerCase();
                card.style.display = text.includes(query.toLowerCase()) ? 'flex' : 'none';
            }});
        }}

        function filterMenu(type) {{
            document.querySelectorAll('.filter-btn').forEach(btn => btn.classList.remove('active'));
            event.target.classList.add('active');
            document.querySelectorAll('.menu-card').forEach(card => {{
                let show = false;
                const calories = parseInt(card.dataset.calories) || 0;
                const price = parseInt(card.dataset.price) || 0;
                const category = card.dataset.category || '';
                if (type === 'all') {{
                    show = true;
                }} else if (type === 'low-cal' && calories < 200) {{
                    show = true;
                }} else if (type === 'veg' && category === 'veg') {{
                    show = true;
                }} else if (type === 'high-price' && price > 30000) {{
                    show = true;
                }}
                card.style.display = show ? 'flex' : 'none';
            }});
        }}

        function showDetail(card) {{
            const name = card.dataset.name;
            const price = formatPrice(card.dataset.price);
            const calories = card.dataset.calories;
            const quantity = card.dataset.quantity;
            const ingredients = card.dataset.ingredients;
            const prepTime = card.dataset.prepTime;
            const difficulty = card.dataset.difficulty;
            const nutrition = card.dataset.nutrition;
            const origin = card.dataset.origin;
            const pairing = card.dataset.pairing;
            const story = card.dataset.story;
            const imgSrc = card.querySelector('img').src;

            document.getElementById('detailImage').src = imgSrc;
            document.getElementById('detailName').textContent = name;
            document.getElementById('detailPrice').textContent = price;
            document.getElementById('detailCalories').textContent = calories;
            document.getElementById('detailQuantity').textContent = quantity;
            document.getElementById('detailIngredients').textContent = ingredients;
            document.getElementById('detailPrepTime').textContent = prepTime;
            document.getElementById('detailDifficulty').textContent = difficulty;
            document.getElementById('detailNutrition').textContent = nutrition;
            document.getElementById('detailOrigin').textContent = origin;
            document.getElementById('detailPairing').textContent = pairing;
            document.getElementById('detailStory').textContent = story;
            document.getElementById('detailAddBtn').onclick = () => addToCartFromMenu(event, card);
        }}

        function addToCartFromMenu(event, card) {{
            event.stopPropagation();
            lastSource = 'menu';
            const name = card.dataset.name;
            const price = parseInt(card.dataset.price);
            cart.push({{name, price, qty: 1, source: 'Menu'}});
            updateCart();
            alert(`✅ Đã thêm ${{name}} vào giỏ!`);
            showTab('cart');
        }}

        document.addEventListener('DOMContentLoaded', function() {{
            // Optional: Show detail for first item by default
            const firstCard = document.querySelector('.menu-card');
            if (firstCard) showDetail(firstCard);
            initSlideshow();
            filterMenu('all'); // Mặc định lọc tất cả
        }});

        function updateCart() {{
            const list = document.getElementById('cartItems');
            const totalEl = document.getElementById('cartTotal');
            list.innerHTML = '';
            let total = 0;
            cart.forEach((item, index) => {{
                const itemTotal = item.price * item.qty;
                const li = document.createElement('li');
                li.className = 'cart-item';
                li.innerHTML = `
                    <img src="${{foodImages[item.name] || 'https://drive.google.com/uc?export=view&id=REPLACE_ID_DEFAULT'}}" alt="${{item.name}}">
                    <div class="cart-item-info">
                        <div>${{item.name}} x${{item.qty}} ${{item.source ? `(${{item.source}})` : ''}}</div>
                        <div class="price">${{formatPrice(itemTotal)}}</div>
                    </div>
                    <button onclick="removeFromCart(${{index}});" style="background: linear-gradient(135deg, #f093fb, #764ba2); color: white; border: none; padding: 10px 15px; border-radius: 25px; font-size: 13px; cursor: pointer; box-shadow: 0 4px 15px rgba(240, 147, 251, 0.3);">Xóa</button>
                `;
                list.appendChild(li);
                total += itemTotal;
            }});
            totalEl.textContent = `Tổng: ${{formatPrice(total)}}`;
            localStorage.setItem('cart', JSON.stringify(cart));
        }}

        function removeFromCart(index) {{
            cart.splice(index, 1);
            updateCart();
        }}

        function checkout() {{
            if (cart.length === 0) {{
                alert('Giỏ hàng trống!');
                return;
            }}
            const total = cart.reduce((sum, i) => sum + i.price * i.qty, 0);
            alert('✅ Đã đặt hàng thành công! Tổng: ' + formatPrice(total));
            saveToHistory(cart, total);
            cart = [];
            updateCart();
            showTab('history');
        }}

        function handlePayment(method) {{
            const total = cart.reduce((sum, i) => sum + i.price * i.qty, 0);
            if (method === 'qr') {{
                showQR(total, true);
            }} else if (method === 'credit') {{
                alert('Thanh toán bằng thẻ tín dụng: Vui lòng nhập thông tin thẻ (mock). Tổng: ' + formatPrice(total));
            }} else if (method === 'cash') {{
                alert('Thanh toán bằng tiền mặt: Vui lòng trả tại quầy. Tổng: ' + formatPrice(total));
            }}
            saveToHistory(cart, total);
            cart = [];
            updateCart();
        }}

        function showQR(total, isModal = true) {{
            const orderId = Date.now();
            const qrData = `THANHTOAN|${{orderId}}|${{total}}|FoodApp`;
            const qrUrl = `https://api.qrserver.com/v1/create-qr-code/?size=200x200&data=${{encodeURIComponent(qrData)}}`;
            if (isModal) {{
                document.getElementById('modalQRCode').src = qrUrl;
                document.getElementById('modalQRAmount').textContent = formatPrice(total);
                document.getElementById('qrModal').style.display = 'block';
            }} else {{
                document.getElementById('qrCode').src = qrUrl;
                document.getElementById('qrAmount').textContent = formatPrice(total);
                document.getElementById('qrSection').classList.add('active');
            }}
        }}

        function closeQRModal() {{
            document.getElementById('qrModal').style.display = 'none';
        }}

        function completePayment(source) {{
            if (source === 'recog') {{
                document.getElementById('qrSection').classList.remove('active');
                resetRecognition();
                showTab('recog');
            }} else {{
                closeQRModal();
                showTab('menu');
            }}
        }}

        function saveToHistory(items, total) {{
            const order = {{
                id: Date.now(),
                date: new Date().toLocaleDateString('vi-VN'),
                items: items,
                total: total
            }};
            let history = JSON.parse(localStorage.getItem('history')) || [];
            history.unshift(order);
            localStorage.setItem('history', JSON.stringify(history));
        }}

        function loadHistory() {{
            const historyList = document.getElementById('historyItems');
            let history = JSON.parse(localStorage.getItem('history')) || [];
            historyList.innerHTML = '';
            history.forEach(order => {{
                const li = document.createElement('li');
                li.className = 'history-item';
                const itemNames = Array.isArray(order.items)
                    ? order.items.map(i => (i.food_name || i.name) + ' x' + (i.qty || 1)).join(', ')
                    : 'Đơn hàng';
                li.innerHTML = `
                    <img src="https://drive.google.com/uc?export=view&id=REPLACE_ID_KHAY" alt="Khay cũ">
                    <div>
                        <h4>Đơn #${{order.id}} - ${{order.date}}</h4>
                        <p>${{itemNames}} - ${{formatPrice(order.total)}} - Đã giao</p>
                    </div>
                    <span class="reorder" onclick="reorderHistory(${{order.id}})">Ăn lại hôm nay</span>
                `;
                historyList.appendChild(li);
            }});
            if (history.length === 0) {{
                historyList.innerHTML = '<div style="text-align: center; color: rgba(255,255,255,0.9); font-size: 15px; padding: 50px; background: rgba(255,255,255,0.1); backdrop-filter: blur(10px); border-radius: 20px; box-shadow: 0 8px 32px rgba(0,0,0,0.1); border: 1px solid rgba(255,255,255,0.2);">Chưa có đơn hàng nào. Hãy khám phá menu!</div>';
            }}
        }}

        function reorderHistory(id) {{
            // Mock reorder
            alert(`Đã thêm đơn hàng cũ #${{id}} vào giỏ!`);
        }}

        function formatPrice(price) {{
            return new Intl.NumberFormat('vi-VN').format(price) + ' VNĐ';
        }}

        updateCart();

        // THÊM CHỨC NĂNG MỚI VÀ CẢI THIỆN (KHÔNG XÓA CODE CŨ)

        // Slideshow in Home
        function initSlideshow() {{
            const slides = document.querySelectorAll('.slide');
            let index = 0;
            setInterval(() => {{
                slides.forEach(slide => slide.classList.remove('active'));
                slides[index].classList.add('active');
                index = (index + 1) % slides.length;
            }}, 3000);
        }}

        // Home Search Suggestions
        function showSuggestions(query) {{
            const suggestions = document.getElementById('homeSuggestions');
            if (query.toLowerCase().includes('cơm')) {{
                suggestions.innerHTML = `
                    <div onclick="searchMenu('Cơm Chiên')">Cơm Chiên</div>
                    <div onclick="searchMenu('Cơm Tấm')">Cơm Tấm</div>
                    <div onclick="searchMenu('Cơm Trứng Chiên')">Cơm Trứng Chiên</div>
                `;
                suggestions.style.display = 'block';
            }} else {{
                suggestions.style.display = 'none';
            }}
        }}

        // Custom Tray
        function openCustomTray() {{
            alert('Mở tùy chỉnh khay cơm ảo (drag & drop)');
            // Có thể mở modal hoặc phần AR tray
        }}

        // AI Chef
        function openAIChef() {{
            document.getElementById('aiChefModal').style.display = 'block';
        }}
        function closeAIChef() {{
            document.getElementById('aiChefModal').style.display = 'none';
        }}
        function sendAIQuestion() {{
            const input = document.getElementById('aiInput').value;
            const chat = document.getElementById('aiChat');
            chat.innerHTML += `<div>Bạn: ${{input}}</div>`;
            // Mock AI response
            setTimeout(() => {{
                chat.innerHTML += `<div>AI: Gợi ý món ít dầu: Rau Xào (100 calo).</div>`;
            }}, 1000);
            document.getElementById('aiInput').value = '';
        }}

        // AR Tray Drag & Drop
        const trayItems = document.querySelectorAll('.tray-item');
        trayItems.forEach(item => {{
            item.addEventListener('dragstart', (e) => {{
                e.dataTransfer.setData('text', e.target.id);
            }});
        }});
        const trayGrid = document.getElementById('trayGrid');
        trayGrid.addEventListener('dragover', (e) => e.preventDefault());
        trayGrid.addEventListener('drop', (e) => {{
            const data = e.dataTransfer.getData('text');
            e.target.appendChild(document.getElementById(data));
            // Cập nhật calo (mock)
            document.getElementById('trayCalo').textContent = Math.random() * 500;
        }});

        // Taste Journal
        function addTasteNote() {{
            const input = document.getElementById('tasteInput').value;
            const timeline = document.getElementById('tasteTimeline');
            timeline.innerHTML += `<li>${{new Date().toLocaleString()}}: ${{input}}</li>`;
            document.getElementById('tasteInput').value = '';
        }}

        // Dark/Light Mode
        function toggleTheme() {{
            document.body.classList.toggle('light-mode');
        }}

        // Compare Dishes
        function openCompare() {{
            document.getElementById('compareModal').style.display = 'block';
        }}
        function closeCompare() {{
            document.getElementById('compareModal').style.display = 'none';
        }}
        function compareDishes() {{
            alert('So sánh hoàn tất!');
        }}

        // Open New Tab (Khắc phục bằng cách hướng dẫn copy URL vì target=_blank có thể bị chặn trong iframe/Colab)
        function openNewTab() {{
            const url = '{public_url}';
            const newTab = window.open(url, '_blank');
            if (!newTab || newTab.closed || typeof newTab.closed == 'undefined') {{
                alert('Mở tab mới bị chặn. Hãy copy URL sau và paste vào tab mới: ' + url);
                prompt("Copy URL này:", url);
            }}
        }}

        // Chatbox FAQ
        function openChatModal() {{
            document.getElementById('chatModal').style.display = 'block';
        }}
        function closeChatModal() {{
            document.getElementById('chatModal').style.display = 'none';
        }}
        function toggleAnswer(index) {{
            const answers = document.querySelectorAll('.faq-answer');
            answers[index].style.display = answers[index].style.display === 'block' ? 'none' : 'block';
        }}

    </script>
</body>
</html>
"""

display(HTML(html_code))

